In [ ]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import RandomOverSampler 
from imblearn.combine import SMOTETomek, SMOTEENN
import joblib

import os

# Defining the path for the csv file
path = os.path.join("dataset.csv")

# Storing the dataframe in a variable named dataset
dataset = pd.read_csv(path)

# Dropping the unnecessary columns
dataset = dataset.drop('seqn', axis='columns')
dataset = dataset.drop('Marital', axis='columns')
print(dataset.shape)
print(len(dataset[dataset['MetabolicSyndrome'] == 0]))
print(dataset.head())

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Defining the columns that need scaling
columns_to_scale = ['Age', 'Income', 'WaistCirc', 'BMI', 'Albuminuria', 
                    'UrAlbCr', 'UricAcid', 'BloodGlucose', 'HDL', 'Triglycerides']

# Creating a MinMaxScaler instance
scaler = MinMaxScaler()

# Scaling the selected columns
dataset[columns_to_scale] = scaler.fit_transform(dataset[columns_to_scale])

# Check the transformed dataset
print(dataset.head())


In [ ]:
sex_mapping = {'Male': 0, 'Female': 1}
race_mapping = {'White': 0, 'Asian': 1, 'Black': 2, 'MexAmerican': 3, 'Hispanic': 4, 'Other': 5}

dataset['Sex'] = dataset['Sex'].replace(sex_mapping)
dataset['Race'] = dataset['Race'].replace(race_mapping)

# This is the incorrect implementation
'''
dataset = dataset.fillna(2)
dataset = dataset.fillna(4)
dataset = dataset.fillna(5)
'''
# Fill NaN values in column with index 2
dataset.iloc[:, 2] = dataset.iloc[:, 2].fillna(dataset.iloc[:, 2].mean())

# Fill NaN values in column with index 4
dataset.iloc[:, 4] = dataset.iloc[:, 4].fillna(dataset.iloc[:, 4].mean())

# Fill NaN values in column with index 5
dataset.iloc[:, 5] = dataset.iloc[:, 5].fillna(dataset.iloc[:, 5].mean())
print(dataset.head())

In [4]:
outcome_0 = dataset[dataset['MetabolicSyndrome'] == 0]
outcome_1 = dataset[dataset['MetabolicSyndrome'] == 1]


test_size_each_class = 400
test_0 = outcome_0.sample(n=test_size_each_class, random_state=42)
test_1 = outcome_1.sample(n=test_size_each_class, random_state=42)
#print(test_1)
test_data = pd.concat([test_0, test_1])

# Remove the test set rows from the original dataset to create the training set
train_data = dataset.drop(test_data.index)


x_train = train_data.drop('MetabolicSyndrome', axis=1).values
y_train = train_data['MetabolicSyndrome'].values
x_test = test_data.drop('MetabolicSyndrome', axis=1).values
y_test = test_data['MetabolicSyndrome'].values
from imblearn.over_sampling import SMOTE, ADASYN
from ctgan import CTGAN
adasyn = ADASYN()
x_resampled, y_resampled = adasyn.fit_resample(x_train, y_train)

In [5]:
classifier_xgboost = XGBClassifier(n_estimators=100, learning_rate=0.3, max_depth=3, random_state=42)
classifier_xgboost.fit(x_resampled, y_resampled)
y_pred_xgboost = classifier_xgboost.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_xgboost)
f1 = f1_score(y_test, y_pred_xgboost)
recall = recall_score(y_test, y_pred_xgboost)
precision = precision_score(y_test, y_pred_xgboost)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(confusion_matrix(y_test, y_pred_xgboost))